<a href="https://colab.research.google.com/github/KaliBond/CAMSCAN/blob/main/claudecams.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Enhanced CAMS-CAN v4.0 - Complete Implementation
# Fully implements the mathematical formulation with all dynamics

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import tensorflow as tf
import networkx as nx
from google.colab import files
import io
from scipy.integrate import odeint
from scipy.stats import entropy
import warnings
import json # Import json for saving insights

warnings.filterwarnings('ignore')

# ============= CORE CAMS MATHEMATICAL MODEL =============

class CAMSNode:
    """Represents a single node in the CAMS network"""
    def __init__(self, node_id, coherence, capacity, stress, abstraction, memory=0):
        self.id = node_id
        self.coherence = coherence
        self.capacity = capacity
        self.stress_chronic = stress * 0.7  # Decompose stress
        self.stress_acute = stress * 0.3
        self.abstraction = abstraction
        self.memory = memory
        self.threshold = 5.0  # Base threshold θ_i0

    def calculate_effective_threshold(self):
        """θ_i(t) = θ_i0 + η_σ * σ_i(t) - η_χ * χ_i(t)"""
        eta_sigma = 0.3  # stress raises threshold
        eta_chi = 0.2    # coherence lowers threshold
        total_stress = self.stress_chronic + self.stress_acute
        return self.threshold + eta_sigma * total_stress - eta_chi * self.coherence

    def neuromodulated_input(self, weights, inputs, beta=0.5, alpha_acute=2.0):
        """u_i(t) = Σ w_ij * s_j - θ_i + β_i(σ_ch + α_acute * σ_ac)"""
        weighted_sum = np.sum(weights * inputs)
        threshold = self.calculate_effective_threshold()
        stress_modulation = beta * (self.stress_chronic + alpha_acute * self.stress_acute)
        return weighted_sum - threshold + stress_modulation

    def activation(self, u):
        """a_i(t) = σ_act(u_i(t)) using sigmoid"""
        return 1 / (1 + np.exp(-u))

class CAMSNetwork:
    """Complete CAMS network implementation with full dynamics"""

    def __init__(self, nodes_data):
        self.nodes = {}
        self.bond_matrix = None
        self.weight_matrix = None
        self.time = 0
        self.dt = 0.1

        # Initialize nodes
        for node_data in nodes_data:
            node = CAMSNode(
                node_data['Node'],
                node_data['Coherence'],
                node_data['Capacity'],
                node_data['Stress'],
                node_data['Abstraction']
            )
            self.nodes[node.id] = node

        # Initialize matrices
        self.initialize_networks()

    def initialize_networks(self):
        """Initialize bond and weight matrices"""
        n = len(self.nodes)
        node_ids = list(self.nodes.keys())

        # Bond matrix B_ij
        self.bond_matrix = np.ones((n, n)) * 0.5
        np.fill_diagonal(self.bond_matrix, 0)

        # Weight matrix w_ij
        self.weight_matrix = np.random.randn(n, n) * 0.1
        np.fill_diagonal(self.weight_matrix, 0)

        # Node mapping
        self.node_to_idx = {node_id: i for i, node_id in enumerate(node_ids)}
        self.idx_to_node = {i: node_id for i, node_id in enumerate(node_ids)}

    def update_capacity(self, node, r_kappa=0.1, phi_kappa=0.05, xi_kappa=0.2, zeta_kappa=0.1):
        """dκ_i/dt = r_κ * a_i + φ_κ * Σ B_ij(κ_j - κ_i) - ξ_κ * σ_i - ζ_κ * C(t)"""
        idx = self.node_to_idx[node.id]

        # Calculate activation
        inputs = np.array([self.nodes[self.idx_to_node[j]].capacity
                          for j in range(len(self.nodes))])
        u = node.neuromodulated_input(self.weight_matrix[idx], inputs)
        a_i = node.activation(u)

        # Bond diffusion term
        bond_diffusion = 0
        for j, other_id in enumerate(self.idx_to_node.values()):
            if other_id != node.id:
                other = self.nodes[other_id]
                bond_diffusion += self.bond_matrix[idx, j] * (other.capacity - node.capacity)

        # Complexity load
        complexity = np.sum(np.abs(self.bond_matrix))

        # Total stress
        total_stress = node.stress_chronic + node.stress_acute

        # Update equation
        dkappa_dt = (r_kappa * a_i +
                    phi_kappa * bond_diffusion -
                    xi_kappa * total_stress -
                    zeta_kappa * complexity)

        return dkappa_dt

    def update_coherence(self, node, lambda_chi=0.05, mu_chi=0.1, rho_chi=0.15):
        """dχ_i/dt = -λ_χ * χ_i + μ_χ * maint_i - Σ B_ij(χ_i - χ_j) - ρ_χ * σ_i * χ_i"""
        idx = self.node_to_idx[node.id]

        # Maintenance (simplified as capacity-based)
        maintenance = node.capacity * 0.1

        # Bond diffusion
        bond_diffusion = 0
        for j, other_id in enumerate(self.idx_to_node.values()):
            if other_id != node.id:
                other = self.nodes[other_id]
                bond_diffusion += self.bond_matrix[idx, j] * (node.coherence - other.coherence)

        # Total stress
        total_stress = node.stress_chronic + node.stress_acute

        # Update equation
        dchi_dt = (-lambda_chi * node.coherence +
                   mu_chi * maintenance -
                   bond_diffusion -
                   rho_chi * total_stress * node.coherence)

        return dchi_dt

    def update_stress(self, node, lambda_sigma=0.1, rho_sigma=0.2, omega_sigma=0.1, upsilon_sigma=0.05):
        """dσ_i/dt = -λ_σ * σ_i + ρ_σ * Σ B_ij(σ_j - σ_i) + ε_i + υ_σ * M_i - ω_σ * a_i"""
        idx = self.node_to_idx[node.id]

        # Calculate activation
        inputs = np.array([self.nodes[self.idx_to_node[j]].capacity
                          for j in range(len(self.nodes))])
        u = node.neuromodulated_input(self.weight_matrix[idx], inputs)
        a_i = node.activation(u)

        # Stress diffusion
        stress_diffusion = 0
        for j, other_id in enumerate(self.idx_to_node.values()):
            if other_id != node.id:
                other = self.nodes[other_id]
                total_stress_j = other.stress_chronic + other.stress_acute
                total_stress_i = node.stress_chronic + node.stress_acute
                stress_diffusion += self.bond_matrix[idx, j] * (total_stress_j - total_stress_i)

        # Memory load
        memory_load = node.memory * upsilon_sigma

        # Random shock (small)
        shock = np.random.normal(0, 0.01)

        # Update equation
        total_stress = node.stress_chronic + node.stress_acute
        dsigma_dt = (-lambda_sigma * total_stress +
                     rho_sigma * stress_diffusion +
                     shock +
                     memory_load -
                     omega_sigma * a_i)

        return dsigma_dt

    def update_abstraction(self, node, gamma_alpha=0.05, delta_alpha=0.1):
        """dα_i/dt = γ_α * M_i - δ_α * σ_i"""
        total_stress = node.stress_chronic + node.stress_acute
        dalpha_dt = gamma_alpha * node.memory - delta_alpha * total_stress
        return dalpha_dt

    def update_memory(self, node):
        """M_i(t+dt) = M_i(t) + dt * α_i * χ_i"""
        return node.abstraction * node.coherence * self.dt

    def calculate_system_health(self):
        """H(t) = Σ w_i * h_i(t) * (1 - P_t)"""
        health_sum = 0
        total_stress = 0
        coherence_values = []

        for node in self.nodes.values():
            # Calculate denominator D_i
            idx = self.node_to_idx[node.id]
            bond_sum = np.sum(self.bond_matrix[idx])
            stress_weighted = node.stress_chronic + 2 * node.stress_acute
            D_i = (1 + bond_sum) * (stress_weighted + 0.01)  # Avoid division by zero

            # Base node output h_i
            h_i = (node.coherence * node.capacity * (1 + 0.5 * node.abstraction)) / D_i

            # Weight (use bond strength as proxy)
            weight = np.mean(self.bond_matrix[idx])
            health_sum += weight * h_i

            total_stress += node.stress_chronic + node.stress_acute
            coherence_values.append(node.coherence)

        # Calculate penalty term (Coherence Asymmetry)
        ca = np.std(coherence_values) / (np.mean(coherence_values) + 0.01)
        stress_tilt = total_stress / (sum(coherence_values) + 0.01)
        P_t = min(ca * stress_tilt, 0.75)

        return health_sum * (1 - P_t)

    def calculate_coherence_entropy(self):
        """E_χ(t) = -Σ χ_i * log(χ_i)"""
        coherence_values = np.array([node.coherence for node in self.nodes.values()])
        # Normalize to probabilities
        coherence_probs = coherence_values / (np.sum(coherence_values) + 0.01)
        coherence_probs = np.clip(coherence_probs, 1e-10, 1)  # Avoid log(0)
        return -np.sum(coherence_probs * np.log(coherence_probs))

    def calculate_network_synchronization(self):
        """S_sync(t) = 1/n² * Σ |dN_i/dt - dN_j/dt|"""
        derivatives = []
        for node in self.nodes.values():
            # Simplified: use capacity change as proxy for dN/dt
            dN_dt = self.update_capacity(node)
            derivatives.append(dN_dt)

        n = len(derivatives)
        sync_sum = 0
        for i in range(n):
            for j in range(n):
                sync_sum += abs(derivatives[i] - derivatives[j])

        return sync_sum / (n * n)

    def calculate_stress_variance(self):
        """V_σ(t) = 1/n * Σ (σ_i - σ_mean)²"""
        stress_values = [node.stress_chronic + node.stress_acute
                        for node in self.nodes.values()]
        return np.var(stress_values)

    def calculate_legitimacy(self, beta1=0.4, beta2=0.3, beta3=0.3):
        """L(t) = β_1 * H(t) + β_2 * I(t) + β_3 * χ_mean"""
        health = self.calculate_system_health()

        # Information integration I(t) = η_I * Σ α_i * κ_i
        info = sum(node.abstraction * node.capacity for node in self.nodes.values()) * 0.1

        # Mean coherence
        chi_mean = np.mean([node.coherence for node in self.nodes.values()])

        return beta1 * health + beta2 * info + beta3 * chi_mean

    def step(self):
        """Perform one time step of the simulation"""
        # Store updates to apply simultaneously
        updates = {}

        for node_id, node in self.nodes.items():
            updates[node_id] = {
                'capacity': self.update_capacity(node),
                'coherence': self.update_coherence(node),
                'stress': self.update_stress(node),
                'abstraction': self.update_abstraction(node),
                'memory': self.update_memory(node)
            }

        # Apply updates
        for node_id, node_updates in updates.items():
            node = self.nodes[node_id]
            node.capacity += node_updates['capacity'] * self.dt
            node.coherence += node_updates['coherence'] * self.dt

            # Update stress (split between chronic and acute)
            stress_change = node_updates['stress'] * self.dt
            node.stress_chronic += stress_change * 0.7
            node.stress_acute += stress_change * 0.3

            node.abstraction += node_updates['abstraction'] * self.dt
            node.memory += node_updates['memory']

            # Bound values
            node.capacity = np.clip(node.capacity, 0, 10)
            node.coherence = np.clip(node.coherence, 0, 10)
            node.stress_chronic = np.clip(node.stress_chronic, 0, 10)
            node.stress_acute = np.clip(node.stress_acute, 0, 10)
            node.abstraction = np.clip(node.abstraction, 0, 10)

        self.time += self.dt

    def simulate(self, steps=100):
        """Run simulation for multiple steps"""
        history = []
        for _ in range(steps):
            self.step()
            state = {
                'time': self.time,
                'health': self.calculate_system_health(),
                'entropy': self.calculate_coherence_entropy(),
                'synchronization': self.calculate_network_synchronization(),
                'stress_variance': self.calculate_stress_variance(),
                'legitimacy': self.calculate_legitimacy()
            }
            history.append(state)
        return history

# ============= DATA PROCESSING & ANALYSIS =============

def load_and_validate_data():
    """Load CSV data with validation"""
    print("=" * 60)
    print("📁 Please upload your CAMS CSV file.")
    print("=" * 60)
    uploaded = files.upload()
    if not uploaded:
        raise ValueError("No file uploaded")

    df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))

    # Validate columns
    required = ['Society', 'Year', 'Node', 'Coherence', 'Capacity', 'Stress', 'Abstraction']
    missing = [col for col in required if col not in df.columns]
    if missing:
        # Try alternative column names
        if 'Nation' in df.columns and 'Society' not in df.columns:
            df['Society'] = df['Nation']
        else:
            raise ValueError(f"Missing required columns: {', '.join(missing)}")


    # Clean data
    df = df.dropna()

    # Calculate Node Value and Bond Strength if missing
    if 'Node Value' not in df.columns:
        df['Node Value'] = df['Coherence'] + df['Capacity'] - df['Stress'] + 0.5 * df['Abstraction']

    if 'Bond Strength' not in df.columns:
        # Simplified bond strength calculation
        df['Bond Strength'] = ((df['Coherence'] + df['Capacity']) * 0.6 +
                               df['Abstraction'] * 0.4) / (1 + np.abs(df['Stress']))

    print(f"✅ Loaded data for {df['Society'].iloc[0]}")
    print(f"📊 Years: {df['Year'].min()}-{df['Year'].max()}")
    print(f"🔗 Nodes: {len(df['Node'].unique())} unique nodes")

    return df

def analyze_civilization_type(df):
    """Classify civilization type based on CAMS patterns"""
    # Calculate average metrics
    avg_abstraction = df['Abstraction'].mean()
    avg_capacity = df['Capacity'].mean()
    avg_stress = df['Stress'].abs().mean()
    avg_coherence = df['Coherence'].mean()
    stress_volatility = df.groupby('Year')['Stress'].std().mean()

    # Classification logic
    if avg_abstraction > 7 and avg_capacity > 7:
        civ_type = "Supergiant"
        description = "High abstraction and capacity, burning bright but potentially unstable"
        color = "#FF6B6B"
    elif avg_coherence > 7 and stress_volatility < 1.5:
        civ_type = "Stable Core"
        description = "Long-lived, highly coherent, conservatively adaptive"
        color = "#4ECDC4"
    elif avg_stress > 6:
        civ_type = "Fragile High-Stress"
        description = "High pressure system, prone to collapse"
        color = "#FF8C42"
    else:
        civ_type = "Resilient Frontier"
        description = "Adaptable, quick to respond to changes"
        color = "#95E77E"

    return {
        'type': civ_type,
        'description': description,
        'color': color,
        'metrics': {
            'avg_abstraction': avg_abstraction,
            'avg_capacity': avg_capacity,
            'avg_stress': avg_stress,
            'avg_coherence': avg_coherence,
            'stress_volatility': stress_volatility
        }
    }

def create_advanced_dashboard(df, network_history=None):
    """Create comprehensive 15-panel dashboard"""
    society = df['Society'].iloc[0]
    years = sorted(df['Year'].unique())

    # Analyze civilization type
    civ_analysis = analyze_civilization_type(df)

    # Create figure
    fig = make_subplots(
        rows=5, cols=3,
        subplot_titles=(
            f"{civ_analysis['type']}: System Health", "Stress Decomposition", "Network Topology",
            "Coherence Entropy", "Phase Space", "Bond Matrix Heatmap",
            "Early Warning Signals", "Capacity Evolution", "Abstraction Dynamics",
            "Stress Propagation", "Legitimacy Index", "Path Dependence",
            "Comparative Resilience", "System Status Indicator", "Future Projections" # Corrected Panel 14 title
        ),
        specs=[
            [{"secondary_y": False}, {"secondary_y": False}, {"type": "scatter3d"}],
            [{"secondary_y": False}, {"secondary_y": False}, {"type": "heatmap"}],
            [{"secondary_y": True}, {"secondary_y": False}, {"secondary_y": False}],
            [{"secondary_y": False}, {"type": "heatmap"}, {"secondary_y": False}], # Corrected Panel 10 type
            [{"type": "bar"}, {"type": "indicator"}, {"secondary_y": False}] # Corrected Panel 13 type
        ],
        vertical_spacing=0.08,
        horizontal_spacing=0.1,
        row_heights=[0.2, 0.2, 0.2, 0.2, 0.2] # Added row heights for better spacing
    )

    # Panel 1: System Health with civilization type
    health_by_year = df.groupby('Year').apply(
        lambda x: (x['Coherence'] * x['Capacity']).sum() / (1 + x['Stress'].abs().sum())
    )
    fig.add_trace(
        go.Scatter(x=years, y=health_by_year, mode='lines+markers',
                  name='System Health', line=dict(color=civ_analysis['color'], width=3)),
        row=1, col=1
    )
    fig.update_yaxes(title_text="System Health", row=1, col=1)


    # Panel 2: Stress Decomposition (Chronic vs Acute)
    stress_chronic_mean = df.groupby('Year')['Stress'].mean() * 0.7
    stress_acute_std = df.groupby('Year')['Stress'].std() * 0.3 # Using std as a proxy for acute stress fluctuation

    fig.add_trace(
        go.Scatter(x=years, y=stress_chronic_mean, name='Chronic Stress (Mean)',
                  line=dict(color='#8B4513')), row=1, col=2
    )
    fig.add_trace(
        go.Scatter(x=years, y=stress_acute_std, name='Acute Stress (Std Dev)', # Clarified label
                  line=dict(color='#FF4500')), row=1, col=2
    )
    fig.update_yaxes(title_text="Stress Level", row=1, col=2)


    # Panel 3: 3D Network Topology
    latest_year = years[-1]
    latest_data = df[df['Year'] == latest_year]

    fig.add_trace(
        go.Scatter3d(
            x=latest_data['Coherence'],
            y=latest_data['Capacity'],
            z=latest_data['Abstraction'],
            mode='markers+text',
            marker=dict(size=latest_data['Node Value']*2,
                       color=latest_data['Stress'],
                       colorscale='RdYlGn_r',
                       showscale=True),
            text=latest_data['Node'],
            name='Nodes'
        ),
        row=1, col=3
    )
    fig.update_layout(scene1 = dict(xaxis_title='Coherence', yaxis_title='Capacity', zaxis_title='Abstraction'))


    # Panel 4: Coherence Entropy
    entropy_values = []
    for year in years:
        year_data = df[df['Year'] == year]
        coherence_vals = year_data['Coherence'].values
        coherence_probs = coherence_vals / (np.sum(coherence_vals) + 1e-10) # Added small epsilon
        entropy_val = entropy(coherence_probs) # Using scipy.stats.entropy
        entropy_values.append(entropy_val)

    fig.add_trace(
        go.Scatter(x=years, y=entropy_values, mode='lines+markers',
                  name='Coherence Entropy', line=dict(color='#9333EA')),
        row=2, col=1
    )
    fig.update_yaxes(title_text="Entropy", row=2, col=1)


    # Panel 5: Phase Space (Coherence vs Stress)
    for node in df['Node'].unique():
        node_data = df[df['Node'] == node]
        fig.add_trace(
            go.Scatter(x=node_data['Coherence'], y=node_data['Stress'],
                      mode='lines', name=node, opacity=0.6),
            row=2, col=2
        )
    fig.update_xaxes(title_text="Coherence", row=2, col=2)
    fig.update_yaxes(title_text="Stress", row=2, col=2)


    # Panel 6: Bond Matrix Heatmap
    nodes = df['Node'].unique()
    n_nodes = len(nodes)
    # Simplified - using a random matrix as a placeholder for actual bond calculation
    bond_matrix = np.random.rand(n_nodes, n_nodes)
    np.fill_diagonal(bond_matrix, 0)

    fig.add_trace(
        go.Heatmap(z=bond_matrix, x=nodes, y=nodes, colorscale='Viridis', name='Bond Strength'),
        row=2, col=3
    )


    # Panel 7: Early Warning Signals (with secondary y-axis)
    variance_rolling = pd.Series(health_by_year).rolling(window=3).std()
    autocorrelation_rolling = pd.Series(health_by_year).rolling(window=3).apply(lambda x: pd.Series(x).autocorr(), raw=True)


    fig.add_trace(
        go.Scatter(x=years, y=health_by_year, name='System Health',
                  line=dict(color='green')),
        row=3, col=1, secondary_y=False
    )
    fig.add_trace(
        go.Scatter(x=years, y=variance_rolling, name='Rolling Variance',
                  line=dict(color='red', dash='dash')),
        row=3, col=1, secondary_y=True
    )
    fig.add_trace(
        go.Scatter(x=years, y=autocorrelation_rolling, name='Rolling Autocorrelation',
                  line=dict(color='purple', dash='dot')),
        row=3, col=1, secondary_y=True
    )
    fig.update_yaxes(title_text="System Health", row=3, col=1, secondary_y=False)
    fig.update_yaxes(title_text="Early Warning Signal Magnitude", row=3, col=1, secondary_y=True)


    # Panel 8: Capacity Evolution
    # Select top 4 nodes based on latest capacity
    latest_capacity = latest_data.set_index('Node')['Capacity'].sort_values(ascending=False)
    top_nodes = latest_capacity.head(4).index.tolist()

    for node in top_nodes:
        node_data = df[df['Node'] == node]
        fig.add_trace(
            go.Scatter(x=node_data['Year'], y=node_data['Capacity'],
                      mode='lines+markers', name=node),
            row=3, col=2
        )
    fig.update_yaxes(title_text="Capacity", row=3, col=2)


    # Panel 9: Abstraction Dynamics
    abstraction_mean = df.groupby('Year')['Abstraction'].mean()
    abstraction_std = df.groupby('Year')['Abstraction'].std()

    fig.add_trace(
        go.Scatter(x=years, y=abstraction_mean, mode='lines',
                  name='Mean Abstraction', line=dict(color='purple')),
        row=3, col=3
    )
    fig.add_trace(
        go.Scatter(x=years, y=abstraction_mean + abstraction_std,
                  mode='lines', line=dict(color='purple', width=0),
                  showlegend=False),
        row=3, col=3
    )
    fig.add_trace(
        go.Scatter(x=years, y=abstraction_mean - abstraction_std,
                  mode='lines', line=dict(color='purple', width=0),
                  fill='tonexty', fillcolor='rgba(147,51,234,0.2)',
                  name='±1 SD'),
        row=3, col=3
    )
    fig.update_yaxes(title_text="Abstraction", row=3, col=3)


    # Panel 10: Stress Propagation Network (using correlation heatmap)
    stress_correlation = df.pivot_table(values='Stress', index='Year', columns='Node').corr()

    fig.add_trace(
        go.Heatmap(z=stress_correlation.values, x=nodes, y=nodes,
                  colorscale='RdBu', zmid=0, name='Stress Correlation'),
        row=4, col=1
    )


    # Panel 11: Legitimacy Index
    legitimacy = []
    for year in years:
        year_data = df[df['Year'] == year]
        # Recalculate health for legitimacy calculation
        health = (year_data['Coherence'] * year_data['Capacity']).sum() / (1 + year_data['Stress'].abs().sum())
        info = (year_data['Abstraction'] * year_data['Capacity']).sum() * 0.1
        coherence_mean = year_data['Coherence'].mean()
        legit = 0.4 * health + 0.3 * info + 0.3 * coherence_mean
        legitimacy.append(legit)

    fig.add_trace(
        go.Scatter(x=years, y=legitimacy, mode='lines+markers',
                  name='Legitimacy', line=dict(color='#059669', width=2)),
        row=4, col=2
    )
    fig.update_yaxes(title_text="Legitimacy Index", row=4, col=2)


    # Panel 12: Path Dependence (using cumulative coherence asymmetry)
    coherence_std_by_year = df.groupby('Year')['Coherence'].std()
    coherence_mean_by_year = df.groupby('Year')['Coherence'].mean()
    coherence_asymmetry = coherence_std_by_year / (coherence_mean_by_year + 1e-10) # Added epsilon
    path_dependence = np.cumprod(1 + coherence_asymmetry.fillna(0).values * 0.01) # Handle potential NaNs

    fig.add_trace(
        go.Scatter(x=years, y=path_dependence, mode='lines+markers',
                  name='Path Dependence', line=dict(color='#7C3AED')),
        row=4, col=3
    )
    fig.update_yaxes(title_text="Path Dependence (Cumulative Asymmetry)", row=4, col=3)


    # Panel 13: Comparative Resilience Score (Bar Chart for recent years)
    resilience_scores = []
    for year in years:
        year_data = df[df['Year'] == year]
        resilience = (year_data['Capacity'].mean() * year_data['Coherence'].mean()) / (year_data['Stress'].abs().mean() + 1e-10) # Added epsilon
        resilience_scores.append(resilience)

    # Show resilience for all years
    fig.add_trace(
        go.Scatter(x=years, y=resilience_scores, mode='lines+markers',
               marker_color='teal', name='Resilience Score'),
        row=5, col=1
    )
    fig.update_yaxes(title_text="Resilience Score", row=5, col=1)


    # Panel 14: System Status Indicator (Gauge)
    current_health = health_by_year.iloc[-1] if not health_by_year.empty else 0
    previous_health = health_by_year.iloc[-2] if len(health_by_year) > 1 else current_health

    status_color = "green" if current_health > 3 else "yellow" if current_health > 1.5 else "red"
    status_text = "Stable" if current_health > 3 else "Warning" if current_health > 1.5 else "Critical"

    fig.add_trace(
        go.Indicator(
            mode="gauge+number+delta",
            value=current_health,
            title={'text': f"System Status: {status_text}"},
            delta={'reference': previous_health, 'relative': True, 'valueformat': ".2f"}, # Added relative delta
            gauge={'axis': {'range': [0, 5]}, # Adjusted range
                   'bar': {'color': status_color},
                   'steps': [
                       {'range': [0, 1.5], 'color': "red"}, # Adjusted steps colors
                       {'range': [1.5, 3], 'color': "yellow"},
                       {'range': [3, 5], 'color': "green"}],
                   'threshold': {'line': {'color': "black", 'width': 2}, # Adjusted threshold appearance
                                'thickness': 0.75, 'value': 1.5}} # Set threshold at 1.5 for Critical
        ),
        row=5, col=2
    )


    # Panel 15: Future Projections (simplified)
    if len(years) > 3:
        # Simple linear projection
        recent_health = health_by_year.iloc[-3:].values
        x = np.arange(3)
        z = np.polyfit(x, recent_health, 1)
        future_years = list(range(years[-1] + 1, years[-1] + 6))
        future_health = [np.polyval(z, i + 3) for i in range(5)]

        fig.add_trace(
            go.Scatter(x=years, y=health_by_year, mode='lines+markers',
                      name='Historical', line=dict(color='blue')),
            row=5, col=3
        )
        fig.add_trace(
            go.Scatter(x=future_years, y=future_health, mode='lines+markers',
                      name='Projected', line=dict(color='orange', dash='dash')),
            row=5, col=3
        )
    fig.update_xaxes(title_text="Year", row=5, col=3)
    fig.update_yaxes(title_text="System Health", row=5, col=3)


    # Update layout
    fig.update_layout(
        height=2000, # Increased height for better spacing
        width=1600,
        title_text=f"<b>{society} - {civ_analysis['type']}</b><br><sup>{civ_analysis['description']}</sup>",
        title_x=0.5,
        showlegend=True,
        hovermode='x unified'
    )


    return fig, civ_analysis

def generate_policy_insights(df, civ_analysis):
    """Generate evidence-based policy recommendations"""
    insights = []

    # Latest year data
    latest_year = df['Year'].max()
    latest_data = df[df['Year'] == latest_year]

    # Historical trends
    years = sorted(df['Year'].unique())
    if len(years) > 5:
        recent_years = years[-5:]
        recent_data = df[df['Year'].isin(recent_years)]

        # Stress trend
        stress_trend = recent_data.groupby('Year')['Stress'].mean()
        if stress_trend.iloc[-1] > stress_trend.iloc[0] * 1.2:
            insights.append({
                'priority': 'HIGH',
                'category': 'Stress Management',
                'insight': 'Stress levels have increased by >20% in recent years',
                'recommendation': 'Implement stress reduction measures: strengthen social safety nets, improve institutional capacity, enhance crisis response systems'
            })

        # Coherence trend
        coherence_trend = recent_data.groupby('Year')['Coherence'].mean()
        if coherence_trend.iloc[-1] < coherence_trend.iloc[0] * 0.8:
            insights.append({
                'priority': 'HIGH',
                'category': 'Social Cohesion',
                'insight': 'Coherence has declined by >20%, indicating growing fragmentation',
                'recommendation': 'Foster inclusive dialogue, strengthen shared institutions, promote common narratives that unite rather than divide'
            })

        # Capacity analysis
        capacity_variance = latest_data['Capacity'].std()
        if capacity_variance > 2:
            insights.append({
                'priority': 'MEDIUM',
                'category': 'Capacity Distribution',
                'insight': 'High variance in capacity across nodes suggests inequality',
                'recommendation': 'Redistribute resources to strengthen weaker nodes, particularly focusing on education and infrastructure'
            })

    # Node-specific insights
    critical_nodes_stress = latest_data[latest_data['Stress'] > 7]['Node'].tolist()
    if critical_nodes_stress:
        insights.append({
            'priority': 'HIGH',
            'category': 'Critical Nodes (Stress)',
            'insight': f"Nodes under critical stress: {', '.join(critical_nodes_stress)}",
            'recommendation': 'Provide targeted support to these sectors to prevent cascade failures'
        })

    # Nodes with low capacity or coherence
    low_capacity_nodes = latest_data[latest_data['Capacity'] < 3]['Node'].tolist()
    low_coherence_nodes = latest_data[latest_data['Coherence'] < 3]['Node'].tolist()

    if low_capacity_nodes:
         insights.append({
            'priority': 'MEDIUM',
            'category': 'Critical Nodes (Capacity)',
            'insight': f"Nodes with low capacity: {', '.join(low_capacity_nodes)}",
            'recommendation': 'Invest in capacity building programs and resource allocation for these nodes.'
        })

    if low_coherence_nodes:
         insights.append({
            'priority': 'MEDIUM',
            'category': 'Critical Nodes (Coherence)',
            'insight': f"Nodes with low coherence: {', '.join(low_coherence_nodes)}",
            'recommendation': 'Focus on improving communication, trust, and shared understanding within and between these nodes.'
        })


    # Abstraction gap
    abstraction_gap = latest_data['Abstraction'].max() - latest_data['Abstraction'].min()
    if abstraction_gap > 4:
        insights.append({
            'priority': 'MEDIUM',
            'category': 'Innovation Gap',
            'insight': 'Large abstraction gap indicates uneven technological/educational development',
            'recommendation': 'Invest in knowledge transfer, education, and innovation infrastructure in lagging sectors'
        })

    # Civilization-type specific recommendations
    if civ_analysis['type'] == 'Supergiant':
        insights.append({
            'priority': 'MEDIUM',
            'category': 'Sustainability',
            'insight': 'Supergiant civilizations risk burnout from high energy expenditure',
            'recommendation': 'Build reserves, improve efficiency, and prepare for controlled deceleration'
        })
    elif civ_analysis['type'] == 'Fragile High-Stress':
        insights.append({
            'priority': 'CRITICAL',
            'category': 'System Stability',
            'insight': 'System operating at dangerous stress levels',
            'recommendation': 'Immediate de-escalation needed: reduce external pressures, strengthen safety valves, improve adaptive capacity'
        })

    return insights

def comparative_analysis(dataframes_dict):
    """Compare multiple civilizations"""
    comparison_results = []

    for name, df in dataframes_dict.items():
        civ_analysis = analyze_civilization_type(df)

        # Calculate key metrics
        latest_year = df['Year'].max()
        latest_data = df[df['Year'] == latest_year]

        result = {
            'Society': name,
            'Type': civ_analysis['type'],
            'Latest_Year': latest_year,
            'System_Health': (latest_data['Coherence'] * latest_data['Capacity']).sum() / (1 + latest_data['Stress'].abs().sum()),
            'Avg_Stress': latest_data['Stress'].abs().mean(),
            'Avg_Coherence': latest_data['Coherence'].mean(),
            'Avg_Capacity': latest_data['Capacity'].mean(),
            'Avg_Abstraction': latest_data['Abstraction'].mean(),
            'Resilience_Score': (latest_data['Capacity'].mean() * latest_data['Coherence'].mean()) / (latest_data['Stress'].abs().mean() + 1)
        }
        comparison_results.append(result)

    comparison_df = pd.DataFrame(comparison_results)

    # Create comparison visualization
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('System Health Comparison', 'Stress vs Resilience',
                       'Civilization Types', 'Capacity-Abstraction Space'),
        specs=[[{"type": "bar"}, {"type": "scatter"}],
               [{"type": "pie"}, {"type": "scatter"}]]
    )

    # System Health Bar Chart
    fig.add_trace(
        go.Bar(x=comparison_df['Society'], y=comparison_df['System_Health'],
               marker_color='lightblue', name='System Health'),
        row=1, col=1
    )

    # Stress vs Resilience Scatter
    fig.add_trace(
        go.Scatter(x=comparison_df['Avg_Stress'], y=comparison_df['Resilience_Score'],
                  mode='markers+text', text=comparison_df['Society'],
                  marker=dict(size=15), name='Societies'),
        row=1, col=2
    )

    # Civilization Types Pie Chart
    type_counts = comparison_df['Type'].value_counts()
    fig.add_trace(
        go.Pie(labels=type_counts.index, values=type_counts.values),
        row=2, col=1
    )

    # Capacity-Abstraction Space
    fig.add_trace(
        go.Scatter(x=comparison_df['Avg_Capacity'], y=comparison_df['Avg_Abstraction'],
                  mode='markers+text', text=comparison_df['Society'],
                  marker=dict(size=comparison_df['System_Health']*10,
                            color=comparison_df['Avg_Stress'],
                            colorscale='RdYlGn_r',
                            showscale=True),
                  name='Societies'),
        row=2, col=2
    )

    fig.update_layout(height=800, width=1200, title_text="Comparative Civilizational Analysis")

    return comparison_df, fig

# ============= MAIN EXECUTION =============

def run_enhanced_cams_analysis():
    """Main execution function"""
    print("=" * 60)
    print("🧠 CAMS-CAN v4.0 - Enhanced Complex Adaptive Analysis")
    print("=" * 60)

    # Load data
    try:
        df = load_and_validate_data()
        society_name = df['Society'].iloc[0]
    except ValueError as e:
        print(f"❌ Data loading failed: {e}")
        print("Please ensure you upload a valid CSV file with the required columns.")
        return None, None, None, None # Return None to indicate failure


    # Run dynamic simulation
    print("\n🔄 Running dynamic simulation...")
    latest_year = df['Year'].max()
    latest_data = df[df['Year'] == latest_year]

    network = CAMSNetwork(latest_data.to_dict('records'))
    simulation_history = network.simulate(steps=50)

    # Create dashboard
    print("📊 Creating advanced dashboard...")
    dashboard, civ_analysis = create_advanced_dashboard(df)

    # Generate insights
    print("\n💡 Generating policy insights...")
    insights = generate_policy_insights(df, civ_analysis)

    # Display results
    print(f"\n🏛️ Civilization Type: {civ_analysis['type']}")
    print(f"📝 {civ_analysis['description']}")

    print("\n📋 Key Policy Insights:")
    # Display all insights
    for insight in insights:
        print(f"\n[{insight['priority']}] {insight['category']}")
        print(f"  → {insight['insight']}")
        print(f"  ✓ {insight['recommendation']}")

    # Calculate final metrics
    # Use the network state after simulation
    final_health = network.calculate_system_health()
    final_entropy = network.calculate_coherence_entropy()
    final_legitimacy = network.calculate_legitimacy()

    print("\n📈 System Metrics (after simulation):") # Clarified metrics are after simulation
    print(f"  • System Health: {final_health:.2f}")
    print(f"  • Coherence Entropy: {final_entropy:.2f}")
    print(f"  • Legitimacy Index: {final_legitimacy:.2f}")
    print(f"  • Network Synchronization: {network.calculate_network_synchronization():.2f}")
    print(f"  • Stress Variance: {network.calculate_stress_variance():.2f}")


    # Show and save dashboard
    dashboard.show()

    # Save results
    dashboard.write_html(f"{society_name}_cams_v4_dashboard.html")
    files.download(f"{society_name}_cams_v4_dashboard.html")

    # Save insights to JSON
    insights_data = {
        'society': society_name,
        'civilization_type': civ_analysis,
        'insights': insights,
        'metrics': {
            'health': final_health,
            'entropy': final_entropy,
            'legitimacy': final_legitimacy,
            'synchronization': network.calculate_network_synchronization(),
            'stress_variance': network.calculate_stress_variance()
        }
    }

    with open(f"{society_name}_insights.json", 'w') as f:
        json.dump(insights_data, f, indent=2, default=str)
    files.download(f"{society_name}_insights.json")

    print("\n✅ Analysis complete! Files saved.")

    return df, network, dashboard, insights

# Execute analysis
if __name__ == "__main__":
    try:
        df, network, dashboard, insights = run_enhanced_cams_analysis()
    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
        print("Please review the error message and your code/data.")

🧠 CAMS-CAN v4.0 - Enhanced Complex Adaptive Analysis
📁 Please upload your CAMS CSV file.


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-2262579481.py", line 973, in <cell line: 0>
    df, network, dashboard, insights = run_enhanced_cams_analysis()
                                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2262579481.py", line 892, in run_enhanced_cams_analysis
    df = load_and_validate_data()
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-2262579481.py", line 325, in load_and_validate_data
    uploaded = files.upload()
               ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/files.py", line 72, in upload
    uploaded_files = _upload_files(multiple=True)
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/google/colab/files.py", line 164, in _upload_files
    result = _outp

TypeError: object of type 'NoneType' has no len()

# Task
Modify the code to load the data from a CSV file located in Google Drive and save the generated dashboard HTML file to Google Drive.

## Mount google drive

### Subtask:
Add a cell to mount Google Drive to the Colab environment.


**Reasoning**:
The first step is to add a cell to mount Google Drive so that files can be accessed from and saved to Drive.



In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Modify data loading

### Subtask:
Update the `load_and_validate_data` function to load the CSV file from a specified path in Google Drive.


**Reasoning**:
The user wants to modify the `load_and_validate_data` function to accept an optional file path for loading data from Google Drive. I will modify the function signature and add conditional logic to load the data using `pd.read_csv` if a file path is provided, while keeping the existing upload functionality as a fallback. I will also update the print statements accordingly.



In [4]:
def load_and_validate_data(file_path=None):
    """Load CSV data with validation, from path or upload"""
    print("=" * 60)
    if file_path:
        print(f"📁 Loading CAMS CSV file from path: {file_path}")
        try:
            df = pd.read_csv(file_path)
            print(f"✅ Loaded data from {file_path}")
        except FileNotFoundError:
            print(f"❌ File not found at {file_path}")
            raise ValueError("Specified file path not found")
        except Exception as e:
            print(f"❌ Error reading file from path: {e}")
            raise ValueError(f"Error reading file: {e}")

    else:
        print("📁 Please upload your CAMS CSV file.")
        try:
            uploaded = files.upload()
            if not uploaded:
                raise ValueError("No file uploaded")
            df = pd.read_csv(io.BytesIO(list(uploaded.values())[0]))
            print("✅ Loaded data from uploaded file")
        except Exception as e:
            print(f"❌ File upload failed: {e}")
            raise ValueError(f"File upload failed: {e}")

    print("=" * 60)

    # Validate columns
    required = ['Society', 'Year', 'Node', 'Coherence', 'Capacity', 'Stress', 'Abstraction']
    missing = [col for col in required if col not in df.columns]
    if missing:
        # Try alternative column names
        if 'Nation' in df.columns and 'Society' not in df.columns:
            df['Society'] = df['Nation']
            missing.remove('Society') # Remove if fixed
        if missing: # Check again after potential fix
             raise ValueError(f"Missing required columns after check: {', '.join(missing)}")


    # Clean data
    df = df.dropna()

    # Calculate Node Value and Bond Strength if missing
    if 'Node Value' not in df.columns:
        df['Node Value'] = df['Coherence'] + df['Capacity'] - df['Stress'] + 0.5 * df['Abstraction']

    if 'Bond Strength' not in df.columns:
        # Simplified bond strength calculation
        df['Bond Strength'] = ((df['Coherence'] + df['Capacity']) * 0.6 +
                               df['Abstraction'] * 0.4) / (1 + np.abs(df['Stress']))

    print(f"📊 Years: {df['Year'].min()}-{df['Year'].max()}")
    print(f"🔗 Nodes: {len(df['Node'].unique())} unique nodes")

    return df

## Modify plot saving

### Subtask:
Update the main execution block to save the generated dashboard HTML file to a specified path in Google Drive.


**Reasoning**:
Modify the main execution function to save the dashboard HTML and insights JSON files to Google Drive paths and update the download paths accordingly.



In [5]:
def run_enhanced_cams_analysis(data_file_path=None, output_dir="/content/drive/My Drive/CAMS_Analysis"):
    """Main execution function"""
    print("=" * 60)
    print("🧠 CAMS-CAN v4.0 - Enhanced Complex Adaptive Analysis")
    print("=" * 60)

    # Ensure output directory exists
    import os
    os.makedirs(output_dir, exist_ok=True)
    print(f"Output will be saved to: {output_dir}")


    # Load data
    try:
        df = load_and_validate_data(file_path=data_file_path)
        society_name = df['Society'].iloc[0]
    except ValueError as e:
        print(f"❌ Data loading failed: {e}")
        print("Please ensure you provide a valid file path or upload a valid CSV file with the required columns.")
        return None, None, None, None # Return None to indicate failure


    # Run dynamic simulation
    print("\n🔄 Running dynamic simulation...")
    latest_year = df['Year'].max()
    latest_data = df[df['Year'] == latest_year]

    # Check if latest_data is empty before proceeding
    if latest_data.empty:
        print("❌ Cannot run simulation: No data available for the latest year.")
        return df, None, None, None


    network = CAMSNetwork(latest_data.to_dict('records'))
    simulation_history = network.simulate(steps=50)

    # Create dashboard
    print("📊 Creating advanced dashboard...")
    dashboard, civ_analysis = create_advanced_dashboard(df)

    # Generate insights
    print("\n💡 Generating policy insights...")
    insights = generate_policy_insights(df, civ_analysis)

    # Display results
    print(f"\n🏛️ Civilization Type: {civ_analysis['type']}")
    print(f"📝 {civ_analysis['description']}")

    print("\n📋 Key Policy Insights:")
    # Display all insights
    for insight in insights:
        print(f"\n[{insight['priority']}] {insight['category']}")
        print(f"  → {insight['insight']}")
        print(f"  ✓ {insight['recommendation']}")

    # Calculate final metrics
    # Use the network state after simulation
    final_health = network.calculate_system_health()
    final_entropy = network.calculate_coherence_entropy()
    final_legitimacy = network.calculate_legitimacy()

    print("\n📈 System Metrics (after simulation):") # Clarified metrics are after simulation
    print(f"  • System Health: {final_health:.2f}")
    print(f"  • Coherence Entropy: {final_entropy:.2f}")
    print(f"  • Legitimacy Index: {final_legitimacy:.2f}")
    print(f"  • Network Synchronization: {network.calculate_network_synchronization():.2f}")
    print(f"  • Stress Variance: {network.calculate_stress_variance():.2f}")


    # Show and save dashboard to Google Drive
    dashboard.show()

    dashboard_path = os.path.join(output_dir, f"{society_name}_cams_v4_dashboard.html")
    dashboard.write_html(dashboard_path)
    print(f"✅ Dashboard saved to {dashboard_path}")
    files.download(dashboard_path) # Download from the Google Drive path


    # Save insights to JSON to Google Drive
    insights_data = {
        'society': society_name,
        'civilization_type': civ_analysis,
        'insights': insights,
        'metrics': {
            'health': final_health,
            'entropy': final_entropy,
            'legitimacy': final_legitimacy,
            'synchronization': network.calculate_network_synchronization(),
            'stress_variance': network.calculate_stress_variance()
        }
    }

    insights_path = os.path.join(output_dir, f"{society_name}_insights.json")
    with open(insights_path, 'w') as f:
        json.dump(insights_data, f, indent=2, default=str)
    print(f"✅ Insights saved to {insights_path}")
    files.download(insights_path) # Download from the Google Drive path


    print("\n✅ Analysis complete! Files saved to Google Drive and downloaded.")

    return df, network, dashboard, insights

# Execute analysis
if __name__ == "__main__":
    try:
        # Example usage: specify the path to your CSV file in Google Drive
        # Replace 'your_folder/your_data.csv' with the actual path
        google_drive_data_path = '/content/drive/My Drive/CAMS_Analysis/sample_cams_data.csv'
        google_drive_output_dir = '/content/drive/My Drive/CAMS_Analysis/Output'

        df, network, dashboard, insights = run_enhanced_cams_analysis(
            data_file_path=google_drive_data_path,
            output_dir=google_drive_output_dir
            )

    except Exception as e:
        print(f"❌ An unexpected error occurred: {e}")
        import traceback
        traceback.print_exc()
        print("Please review the error message and your code/data.")

🧠 CAMS-CAN v4.0 - Enhanced Complex Adaptive Analysis
Output will be saved to: /content/drive/My Drive/CAMS_Analysis/Output
📁 Loading CAMS CSV file from path: /content/drive/My Drive/CAMS_Analysis/sample_cams_data.csv
❌ File not found at /content/drive/My Drive/CAMS_Analysis/sample_cams_data.csv
❌ Data loading failed: Specified file path not found
Please ensure you provide a valid file path or upload a valid CSV file with the required columns.


## Summary:

### Data Analysis Key Findings

*   The execution failed because the specified input data CSV file (`/content/drive/My Drive/CAMS_Analysis/sample_cams_data.csv`) was not found at the given path in Google Drive, preventing the analysis and file saving steps from completing.

### Insights or Next Steps

*   Verify that the input CSV file exists at the specified Google Drive path (`/content/drive/My Drive/CAMS_Analysis/sample_cams_data.csv`) and that the path is correctly typed.
*   Ensure that the Colab notebook has the necessary permissions to access files in the specified Google Drive location.
